In [2]:
# First Step is to obtain the data from the census website
# After some research concluded that B08303 is the correct data set to use (it is titled "Travel Time to Work"
# The basis for using this dataset is the obvious inverse relationship between Telework and Commute Times

In [3]:
# We begin with the 2023 and 2022 "ACS 1-Year Estimates Detailed Tables".
# The weblinks are: https://data.census.gov/table/ACSDT1Y2023.B08303?g=010XX00US$3300000 for 2023
# The link is the census website (data.census.gov), "Table" because we want the option to download
# ACSDT1, I have learned, stands for American Community Survey (ACS) Detailed Tables - perhaps data tables - for 1 year
#  The last 5 year was issued in 2022 so the most recent data in 2023 is single year
# B08303 refers to the "Travel to Work Data" which for historic reasons is known as B08303
# This was then set up by geography - hence the "g=", and then the 010 is a placeholder, and XX00US is a placeholder for all States
#  33000000 appears to be by metropolitan area

In [4]:
# The given the small number of files we need to download this can be done manually
# But to show that it CAN be done automatically used Claude to create some code which will use an API (secret under env)
# to download data for each year and then to create csv files with that data

In [2]:
import os
import time
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
from requests.exceptions import RequestException

# Load environment variables from the specific .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv('CENSUS_API_KEY')

# Define human-readable column names
column_names = {
    'NAME': 'Metro Area',
    'B08303_001E': 'Total',
    'B08303_002E': 'Less than 5 minutes',
    'B08303_003E': '5 to 9 minutes',
    'B08303_004E': '10 to 14 minutes',
    'B08303_005E': '15 to 19 minutes',
    'B08303_006E': '20 to 24 minutes',
    'B08303_007E': '25 to 29 minutes',
    'B08303_008E': '30 to 34 minutes',
    'B08303_009E': '35 to 39 minutes',
    'B08303_010E': '40 to 44 minutes',
    'B08303_011E': '45 to 59 minutes',
    'B08303_012E': '60 to 89 minutes',
    'B08303_013E': '90 or more minutes'
}

def fetch_and_save_data(year, max_retries=3, delay=5):
    # Census API endpoint
    api_url = f"https://api.census.gov/data/{year}/acs/acs1"

    # Parameters for the API request
    params = {
        "get": ",".join(column_names.keys()),
        "for": "metropolitan statistical area/micropolitan statistical area:*",
        "key": api_key
    }

    for attempt in range(max_retries):
        try:
            # Make the API request
            response = requests.get(api_url, params=params, timeout=30)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Process the data
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            
            # Rename columns to human-readable names
            df = df.rename(columns=column_names)
            
            # Convert numeric columns to integers, replacing non-numeric values with NaN
            numeric_columns = df.columns[1:-1]  # All columns except 'Metro Area' and the last one
            for col in numeric_columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Replace NaN with 0 or handle as needed
            df[numeric_columns] = df[numeric_columns].fillna(0).astype(int)
            
            # Remove the metropolitan statistical area/micropolitan statistical area code column
            df = df.drop(columns=['metropolitan statistical area/micropolitan statistical area'])
            
            # Add a Year column
            df['Year'] = year
            
            # Save to CSV
            output_file = f'Commute_Time_By_Metro_Area_{year}.csv'
            df.to_csv(output_file, index=False)
            print(f"Data for {year} saved to {os.path.abspath(output_file)}")
            return df

        except RequestException as e:
            print(f"Attempt {attempt + 1} failed for year {year}. Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Failed to fetch data for {year} after {max_retries} attempts.")
                return None

# Years to fetch data for
years = range(2010, 2025)  # 2010 to 2024

all_data = []

for year in years:
    print(f"Fetching data for {year}...")
    year_data = fetch_and_save_data(year)
    if year_data is not None:
        all_data.append(year_data)
    else:
        print(f"Skipping {year} due to persistent errors.")
    print()  # Add a blank line for readability
    time.sleep(2)  # Add a 2-second delay between years to avoid overwhelming the API

# Combine all data into a single DataFrame
if all_data:
    global_df = pd.concat(all_data, ignore_index=True)
    
    # Save the global CSV
    global_csv = 'Commute_Time_By_Metro_Area_All_Years.csv'
    global_df.to_csv(global_csv, index=False)
    print(f"Global data for all years saved to {os.path.abspath(global_csv)}")
else:
    print("No data was successfully retrieved for any year.")

print("Data fetching complete.")

Fetching data for 2010...
Data for 2010 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2010.csv

Fetching data for 2011...
Data for 2011 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2011.csv

Fetching data for 2012...
Data for 2012 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2012.csv

Fetching data for 2013...
Data for 2013 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2013.csv

Fetching data for 2014...
Data for 2014 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2014.csv

Fetching data for 2015...
Data for 2015 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2015.csv

Fetching data for 2016...
Data for 2016 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2016.csv

Fetching data for 2017...
Data for 2017 saved to /Users/baruchgottesman/ai-project-2/Commute_Time_By_Metro_Area_2017.csv

Fetching data for 2018..

In [38]:
import pandas as pd

# Read the CSV file
file_name = 'Commute_Time_By_Metro_Area_All_Years.csv'
df = pd.read_csv(file_name)

# Function to clean up the Metro Area name
def clean_metro_name(name):
    # Split by dash and take the first part
    first_part = name.split('-')[0].strip()
    
    # Handle multi-word city names (e.g., "New York")
    words = first_part.split()
    if len(words) > 1 and words[1].istitle():
        return ' '.join(words[:2])  # Return first two words if second is capitalized
    else:
        return words[0]  # Otherwise, return just the first word

# Apply the cleaning function to the Metro Area column
df['Metro Area'] = df['Metro Area'].apply(clean_metro_name)

# Save the updated DataFrame back to the same CSV file
df.to_csv(file_name, index=False)

print(f"File '{file_name}' has been updated with cleaned Metro Area names.")

# Display the first few rows to verify the changes
print(df[['Metro Area']].head(30))

File 'Commute_Time_By_Metro_Area_All_Years.csv' has been updated with cleaned Metro Area names.
       Metro Area
0       Aberdeen,
1        Abilene,
2         Adrian,
3       Aguadilla
4          Akron,
5         Albany,
6          Albany
7          Albany
8    Albertville,
9    Albuquerque,
10    Alexandria,
11       Allegan,
12      Allentown
13       Altoona,
14      Amarillo,
15          Ames,
16     Anchorage,
17      Anderson,
18      Anderson,
19     Ann Arbor,
20       Anniston
21      Appleton,
22     Asheville,
23     Ashtabula,
24        Athens,
25         Athens
26        Atlanta
27  Atlantic City
28        Auburn,
29         Auburn


In [40]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Metro_Area_All_Years.csv')

# Group by Metro Area and calculate the average Total
metro_averages = df.groupby('Metro Area')['Total'].mean().reset_index()

# Sort by Total in descending order and select top 50
top_50_metros = metro_averages.sort_values('Total', ascending=False).head(50)

# Get the list of top 50 metro areas
top_50_metro_list = top_50_metros['Metro Area'].tolist()

# Filter the original dataframe to include only the top 50 metro areas
top_50_data = df[df['Metro Area'].isin(top_50_metro_list)]

# Sort the data by Metro Area and any other relevant columns (e.g., year if present)
top_50_data = top_50_data.sort_values(['Metro Area'])

# Save the results to a new CSV file
top_50_data.to_csv('Commute_Time_By_Top_50_Metro_Area_All_Years.csv', index=False)

print(f"Data for top 50 metro areas has been saved to 'Commute_Time_By_Top_50_Metro_Area_All_Years.csv'")
print(top_50_data.shape)


Data for top 50 metro areas has been saved to 'Commute_Time_By_Top_50_Metro_Area_All_Years.csv'
(624, 15)


In [42]:

import pandas as pd

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Top_50_Metro_Area_All_Years.csv')

# Define the columns for each new bucket
no_commute_cols = ["Less than 5 minutes"]
yes_commute_cols = [
    "5 to 9 minutes", "10 to 14 minutes", "15 to 19 minutes", "20 to 24 minutes",
    "25 to 29 minutes", "30 to 34 minutes", "35 to 39 minutes", "40 to 44 minutes",
    "45 to 59 minutes", "60 to 89 minutes", "90 or more minutes"
]
short_commute_cols = [
    "5 to 9 minutes", "10 to 14 minutes", "15 to 19 minutes",
    "20 to 24 minutes", "25 to 29 minutes"
]
medium_commute_cols = [
    "30 to 34 minutes", "35 to 39 minutes", "40 to 44 minutes",
    "45 to 59 minutes"
]
long_commute_cols = ["60 to 89 minutes", "90 or more minutes"]

# Create new columns with the summed values
df["No Commute"] = df[no_commute_cols].sum(axis=1)
df["Yes Commute (more than 5 minutes)"] = df[yes_commute_cols].sum(axis=1)
df["Short Commutes - Less than half-hour"] = df[short_commute_cols].sum(axis=1)
df["Medium Commutes - Half-hour to one hour"] = df[medium_commute_cols].sum(axis=1)
df["Long Commutes - More than one hour"] = df[long_commute_cols].sum(axis=1)

# Drop the original commute time columns
columns_to_drop = no_commute_cols + yes_commute_cols
df = df.drop(columns=columns_to_drop)

# Save the transformed dataframe to a new CSV file
df.to_csv('Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets.csv', index=False)
print("Transformation complete. New CSV file created.")
print(top_50_data.shape)

Transformation complete. New CSV file created.
(624, 15)


In [46]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets.csv')

# Define the two separate categories
commute_type_columns = [
    'No Commute',
    'Yes Commute (more than 5 minutes)'
]

commute_duration_columns = [
    'Short Commutes - Less than half-hour',
    'Medium Commutes - Half-hour to one hour',
    'Long Commutes - More than one hour'
]

# Calculate percentages for commute type (No Commute vs Yes Commute)
commute_type_total = df[commute_type_columns].sum(axis=1)
for col in commute_type_columns:
    df[f'{col} (%)'] = (df[col] / commute_type_total * 100).round(5)

# Calculate percentages for commute duration (Short, Medium, Long)
commute_duration_total = df[commute_duration_columns].sum(axis=1)
for col in commute_duration_columns:
    df[f'{col} (%)'] = (df[col] / commute_duration_total * 100).round(5)

# Save the results to a new CSV file
df.to_csv('Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets_Percentage.csv', index=False)

print("Percentage data has been saved to 'Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets_Percentage.csv'")

Percentage data has been saved to 'Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets_Percentage.csv'


In [52]:
import pandas as pd

def clean_metro_name(name):
    first_part = name.split('-')[0].strip()
    words = first_part.split()
    return ' '.join(words[:2]) if len(words) > 1 and words[1].istitle() else words[0]

# Read the CSV file
input_file = 'Commute_Time_By_Top_50_Metro_Area_All_Years_Five_Buckets_Percentage.csv'
output_file = 'Final_Commute_Time.csv'

df = pd.read_csv(input_file)

# Clean up the Metro Area name
df['Metro'] = df['Metro Area'].apply(clean_metro_name)

# Drop the original 'Metro Area' column if it exists
if 'Metro Area' in df.columns:
    df = df.drop(columns=['Metro Area'])

# Reorder columns to put 'Metro' first
cols = ['Metro'] + [col for col in df.columns if col != 'Metro']
df = df[cols]

# Save the updated DataFrame to the new CSV file
df.to_csv(output_file, index=False)

print(f"File '{output_file}' has been created with the cleaned data.")

# Display the first few rows to verify the changes
print(df[['Metro', 'Total', 'Year']].head())

# Display all column names to verify the structure
print("\nAll columns in the updated file:")
print(df.columns.tolist())
print(df.head(600))

File 'Final_Commute_Time.csv' has been created with the cleaned data.
     Metro    Total  Year
0  Atlanta  2232939  2010
1  Atlanta  2715426  2018
2  Atlanta  2588713  2023
3  Atlanta  2533900  2015
4  Atlanta  2264984  2011

All columns in the updated file:
['Metro', 'Total', 'Year', 'No Commute', 'Yes Commute (more than 5 minutes)', 'Short Commutes - Less than half-hour', 'Medium Commutes - Half-hour to one hour', 'Long Commutes - More than one hour', 'No Commute (%)', 'Yes Commute (more than 5 minutes) (%)', 'Short Commutes - Less than half-hour (%)', 'Medium Commutes - Half-hour to one hour (%)', 'Long Commutes - More than one hour (%)']
              Metro    Total  Year  No Commute  \
0           Atlanta  2232939  2010       38502   
1           Atlanta  2715426  2018       32705   
2           Atlanta  2588713  2023       44118   
3           Atlanta  2533900  2015       40298   
4           Atlanta  2264984  2011       38588   
..              ...      ...   ...         ...   